In [1]:
import pandas as pd
import numpy as np
from keras.layers import Dense, SimpleRNN, GRU , LSTM , Embedding
from keras.models  import Sequential

#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical

In [2]:
#pip install numpy == 1.19.5

In [3]:
messages = pd.read_csv(r'D:\Datasets\spam1.csv', encoding = 'cp1252')

In [4]:
messages.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
messages = messages.iloc[:, [0,1]]
messages.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
messages.rename(columns = {'v1': 'label', 'v2':'message'}, inplace = True)
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
messages.label = messages.label.replace({'ham':0, 'spam':1})
messages.head()

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
messages_x = messages.iloc[:, 1]
messages_y = messages.iloc[:, 0]
from sklearn.model_selection import train_test_split

x_train , x_test ,y_train , y_test = train_test_split(messages_x , messages_y , test_size = .2)


In [9]:
y_train = to_categorical(y_train)
#y_test  = to_categorical(y_test)


In [10]:
y_train.shape

(5420, 2)

In [11]:
max_num_words = 10000
seq_len       = 50
embedding_size  = 100

In [12]:
from keras.preprocessing.text import Tokenizer  #  is used for tokinizeation
from keras.preprocessing.sequence import pad_sequences  # used to bring all samples to same length
#from keras.utils import to_categorical #  ( this function will be used to convert the labels to one hot encoding)
import numpy as np
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical

tokenizer = Tokenizer(num_words = max_num_words)
tokenizer.fit_on_texts(messages.message)
x_train = tokenizer.texts_to_sequences(x_train) # will convert the text to sequences of IDs
x_train = pad_sequences(x_train , maxlen = seq_len)


x_test  =  tokenizer.texts_to_sequences(x_test)
x_test  =  pad_sequences(x_test , maxlen = seq_len)


model = Sequential()  # iniitialize the network
model.add(Embedding(input_dim = max_num_words,    # Embedding takes input in sequenses and gives output in embedings
                   input_length = seq_len,
                    output_dim = embedding_size))


In [13]:
type(y_train)
type(x_train)

numpy.ndarray

In [14]:
print(x_train.shape)
print(y_train.shape)

(5420, 50)
(5420, 2)


In [15]:
model.add(LSTM(5)) 
model.add(Dense(2 , activation  = 'softmax'))  # this 2 bcoz u have 2 level in target variable
#from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
adam = Adam(.001)
model.compile(optimizer= adam , loss = 'categorical_crossentropy' , metrics = ['accuracy'])


In [16]:
model.fit(x_train , y_train , epochs = 5 , batch_size= 32 , validation_split= .2)


Epoch 1/5
136/136 [==============================] - 6s 30ms/step - loss: 0.3075 - accuracy: 0.9200 - val_loss: 0.1197 - val_accuracy: 0.9825
Epoch 2/5
136/136 [==============================] - 3s 21ms/step - loss: 0.0748 - accuracy: 0.9894 - val_loss: 0.0606 - val_accuracy: 0.9880
Epoch 3/5
136/136 [==============================] - 3s 19ms/step - loss: 0.0300 - accuracy: 0.9970 - val_loss: 0.0409 - val_accuracy: 0.9917
Epoch 4/5
136/136 [==============================] - 3s 19ms/step - loss: 0.0143 - accuracy: 0.9988 - val_loss: 0.0370 - val_accuracy: 0.9908
Epoch 5/5
136/136 [==============================] - 2s 18ms/step - loss: 0.0077 - accuracy: 0.9998 - val_loss: 0.0362 - val_accuracy: 0.9908


In [17]:
pred = model.predict(x_test)
pred

array([[0.9965647 , 0.00343532],
       [0.99743444, 0.00256556],
       [0.9929744 , 0.00702555],
       ...,
       [0.996117  , 0.00388301],
       [0.01910798, 0.980892  ],
       [0.996774  , 0.00322598]], dtype=float32)

In [18]:
pred_classes = np.argmax(pred, axis=1)
pred_classes

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [19]:
from sklearn.metrics import confusion_matrix

In [20]:
cm = confusion_matrix(y_test, pred_classes )
cm

array([[1150,    4],
       [   6,  196]], dtype=int64)

In [21]:
accuracy = cm.diagonal().sum()/cm.sum()
accuracy

0.9926253687315634